### Objetivo

O objetivo desse secção é explorar os dados de criptomoedas consumidos pela API gratuita da CoinGeck.<br>

Por ser uma análise exploratória utilizaremos a IDE jupyter Notebook para fazer as analises.<br>

As bibliotecas que serão utilizadas:
```python
import pandas as pd
import datetime as dt
import time as t
import plotly.graph_objects as go
from plotly.offline import plot
from pycoingecko import CoinGeckoAPI
```

In [6]:
#import modules
import pandas as pd
import datetime as dt
import time as t
import plotly.graph_objects as go
from plotly.offline import plot
from pycoingecko import CoinGeckoAPI


# create a client for free public API
cg = CoinGeckoAPI()

# connection test
try:
    response = cg.ping()
    if response.get("gecko_says"):
        print(f"Conexão estabelecida com sucesso: {response["gecko_says"]}")
    else:
        print(f"Conexão feita, mas resposta inesperada: {response}")
except Exception as e:
    print("FALHA na conexão: {e}")

Conexão estabelecida com sucesso: (V3) To the Moon!


### Alguns exemplos do uso da CoinGeckAPI

- Endpoint /simple/price com os parâmetros necessários
- listas podem ser usadas para argumentos com múltiplos valores
- listas podem ser usadas para argumentos com múltiplos valores
- Parâmetros opcionais podem ser passados ​​conforme definido na documentação da API (https://www.coingecko.com/api/docs/v3).

In [4]:
# /simple/price endpoint with the required parameters
bitcoin_price = cg.get_price(ids="bitcoin", vs_currencies="usd")
print(f"Bitcoin price in dollars = U$ {bitcoin_price["bitcoin"]["usd"]}")

Bitcoin price in dollars = U$ 91900


Pode ser criada listas que contenham os ids das criptomoedas e assim buscar o valor de cada criptomoeda

In [5]:
cripto_list = ["bitcoin", "litecoin", "ethereum"]
criptos_price = cg.get_price(ids=cripto_list, vs_currencies="usd")
criptos_price

{'bitcoin': {'usd': 91907},
 'ethereum': {'usd': 3067.52},
 'litecoin': {'usd': 95.51}}

Também é possível trazer o buscar o preço das criptomoedas em mais de uma moeda

In [6]:
criptos_price_usd_eur = cg.get_price(ids=cripto_list, vs_currencies="usd,eur")
criptos_price_usd_eur

{'bitcoin': {'usd': 91991, 'eur': 79328},
 'ethereum': {'usd': 3069.77, 'eur': 2647.18},
 'litecoin': {'usd': 95.53, 'eur': 82.38}}

Podemos incluir como parâmetros adicionais
- include market cap  -> incluir capitalização de mercado
- include 24hr vol -> incluir volume de 24 horas
- include 24hr change -> incluir variação de 24 horas
- include last updated at -> incluir última atualização

In [17]:
bitcoin_price_market = cg.get_price(ids="bitcoin", 
                                    vs_currencies="usd", 
                                    include_market_cap="True", 
                                    include_24hr_vol="True", 
                                    include_24hr_change="True", 
                                    include_last_update_at="True")

bitcoin_price_market

{'bitcoin': {'usd': 93111,
  'usd_market_cap': 1856922864223.1934,
  'usd_24h_vol': 124639335259.96385,
  'usd_24h_change': -0.0025403168368015473}}

### Ping

- cg.ping(): Checar o status da API

### Simple
Métodos simples sobre preço das moedas
- cg.get_price(): retorna o preço de crip
- cg.get_token_price(): preço dos token utilizando o endereço de contrato
- cg.get_supported_vs_currencies(): Obtem a lista de moedas suportadas

In [11]:
# Get bitcoin price
cg.get_price(ids="bitcoin", vs_currencies="usd")

{'bitcoin': {'usd': 93356}}

In [ ]:
# Consultar preço do USDT (Tether) na rede Ethereum
token_price = cg.get_token_price(
    id='ethereum',
    contract_addresses='0xdac17f958d2ee523a2206206994597c13d831ec7', # Endereço(s) do contrato do token na blockchain
    vs_currencies='usd',
    include_market_cap=True,
    include_24hr_vol=True,
    include_24hr_change=True,
    include_last_updated_at=True
)

print(token_price)

{'0xdac17f958d2ee523a2206206994597c13d831ec7': {'usd': 0.999338, 'usd_market_cap': 183911403753.0544, 'usd_24h_vol': 165805960919.453, 'usd_24h_change': -0.008064401857896584, 'last_updated_at': 1763486591}}


In [16]:
# Get list currencies supported
currencies_list = cg.get_supported_vs_currencies(
    id='ethereum',
    vs_currencies='usd')

print(currencies_list)
print(len(currencies_list))

['btc', 'eth', 'ltc', 'bch', 'bnb', 'eos', 'xrp', 'xlm', 'link', 'dot', 'yfi', 'sol', 'usd', 'aed', 'ars', 'aud', 'bdt', 'bhd', 'bmd', 'brl', 'cad', 'chf', 'clp', 'cny', 'czk', 'dkk', 'eur', 'gbp', 'gel', 'hkd', 'huf', 'idr', 'ils', 'inr', 'jpy', 'krw', 'kwd', 'lkr', 'mmk', 'mxn', 'myr', 'ngn', 'nok', 'nzd', 'php', 'pkr', 'pln', 'rub', 'sar', 'sek', 'sgd', 'thb', 'try', 'twd', 'uah', 'vef', 'vnd', 'zar', 'xdr', 'xag', 'xau', 'bits', 'sats']
63


### Coins

- cg.get_coins_list(): Liste o ID, nome e símbolo de todas as moedas suportadas (sem necessidade de paginação).
- cg.get_coins_markets(): Liste todos os preços, capitalização de mercado, volume e dados relacionados ao mercado das moedas suportadas.
- cg.get_coin_by_id(): Obtenha dados atualizados (nome, preço, mercado, ... incluindo códigos de negociação) para uma criptomoeda.
- cg.get_coin_ticker_by_id(): Obtenha listas de cotações de moedas (paginadas até 100 itens)
- cg.get_coin_history_by_id(): Obtenha dados históricos (nome, preço, mercado, estatísticas) de uma moeda em uma data específica.
- cg.get_coin_market_chart_by_id(): Obtenha dados históricos de mercado, incluindo preço, capitalização de mercado e volume de 24 horas (granularidade automática).
- cg.get_coin_market_chart_range_by_id(): Obtenha dados históricos de mercado, incluindo preço, capitalização de mercado e volume de 24 horas, dentro de um intervalo de datas (granularidade automática).
- cg.get_coin_ohlc_by_id(): Obtenha o gráfico OHLC (Abertura, Máxima, Mínima, Fechamento) de uma moeda com base em um ID de moeda específico



####
```python
cg.get_coins_list(include_platform=False)
```

- include_platform (bool, opcional)
    - Se False (padrão): retorna apenas informações básicas das moedas (id, símbolo, nome).
    - Se True: além disso, retorna também os endereços de contrato dos tokens em diferentes blockchains (Ethereum, Binance Smart Chain, Polygon, etc.).

In [ ]:
# Listar todos os ids, nomes e símbolos de todas as moedas
coin_list = cg.get_coins_list()

for item in coin_list:
    print(item)


####
```python
cg.get_coins_markets(vs_currency="usd", ids=None, order='market_cap_desc', per_page=100, page=1, sparkline=False, price_change_percentage=None)
```

- vs_currency (string, obrigatório) → Moeda de referência para preços (ex.: "usd", "eur", "brl").
- ids (string ou lista, opcional) → Lista de moedas específicas (ex.: "bitcoin,ethereum").
- order (string, opcional) → Ordenação dos resultados (ex.: "market_cap_desc", "volume_asc", "id_asc").
- per_page (int, opcional) → Quantidade de resultados por página (máx. 250).
- page (int, opcional) → Número da página para paginação.
- sparkline (bool, opcional) → Se True, inclui dados de sparkline (mini gráfico de preço).
- price_change_percentage (string, opcional) → Inclui variação percentual em intervalos (ex.: "1h,24h,7d").

In [ ]:
cg.get_coins_markets(vs_currency="usd", ids="bitcoin")

####
```python
cg.get_coin_by_id(id, localization=True, tickers=True, market_data=True, community_data=True, developer_data=True, sparkline=False)
```

- id (string, obrigatório) → Identificador da moeda (ex.: "bitcoin", "ethereum").
- localization (bool, opcional) → Se True, retorna traduções/localizações do nome da moeda.
- tickers (bool, opcional) → Se True, inclui informações de pares de negociação (exchanges e pares).
- market_data (bool, opcional) → Se True, retorna dados de mercado (preço, market cap, volume, etc.).
- community_data (bool, opcional) → Se True, inclui dados de comunidade (seguidores, atividade social).
- developer_data (bool, opcional) → Se True, retorna estatísticas de desenvolvimento (commits, forks, issues).
- sparkline (bool, opcional) → Se True, inclui dados de sparkline (mini gráfico de preço).

In [ ]:
# Get data current for a coin by id
cg.get_coin_by_id("bitcoin")

In [24]:
# Consultando informações detalhadas do Bitcoin
bitcoin_data = cg.get_coin_by_id(
    id='bitcoin',
    localization=False,       # não trazer traduções
    tickers=False,            # não incluir pares de negociação
    market_data=True,         # incluir dados de mercado
    community_data=True,      # incluir dados de comunidade
    developer_data=True,      # incluir dados de desenvolvimento
    sparkline=False           # não incluir sparkline
)

# Exemplo de acesso a alguns dados
print("Nome:", bitcoin_data['name'])
print("Símbolo:", bitcoin_data['symbol'])
print("Preço atual em USD:", bitcoin_data['market_data']['current_price']['usd'])
print("Market Cap em USD:", bitcoin_data['market_data']['market_cap']['usd'])
print("Volume 24h em USD:", bitcoin_data['market_data']['total_volume']['usd'])

Nome: Bitcoin
Símbolo: btc
Preço atual em USD: 93653
Market Cap em USD: 1861856861287
Volume 24h em USD: 120019844174


####
```python
cg.get_coin_ticker_by_id(id, exchange_ids=None, include_exchange_logo=False, page=1, order=None, depth=False)
```

- id (string, obrigatório) → Identificador da moeda (ex.: "bitcoin", "ethereum").
- exchange_ids (string ou lista, opcional) → Filtra os tickers por exchanges específicas (ex.: "binance,coinbase").
- include_exchange_logo (bool, opcional) → Se True, inclui o logo da exchange nos resultados.
- page (int, opcional) → Número da página para paginação dos tickers.
- order (string, opcional) → Ordenação dos tickers (ex.: "volume_desc", "trust_score_desc").
- depth (bool, opcional) → Se True, inclui dados de profundidade de mercado (order book).

In [ ]:
# Get coin ticker
cg.get_coin_ticker_by_id(id="bitcoin", exchange_ids="coinbase")

####
```python
cg.get_coin_history_by_id(id, date, localization=True)
```

- id (string, obrigatório) → Identificador da moeda (ex.: "bitcoin", "ethereum").
- date (string, obrigatório) → Data desejada no formato DD-MM-YYYY (ex.: "30-12-2017").
- localization (bool, opcional) → Se True, retorna traduções/localizações do nome da moeda; se False, ignora.

In [ ]:
# Get historical data the of bitcoin
btc_history = cg.get_coin_history_by_id(
    id='bitcoin',
    date='2025-11-15',
    localization=False   # no translation available
)


In [10]:
print("Nome:", btc_history['name'])
print("Símbolo:", btc_history['symbol'])
print("Preço em USD:", btc_history['market_data']['current_price']['usd'])

Nome: Bitcoin
Símbolo: btc
Preço em USD: 94456.39368235337


####
```python
cg.get_coin_market_chart_by_id(id, vs_currency, days, interval=None)
```

- id (string, obrigatório) → Identificador da moeda (ex.: "bitcoin", "ethereum").
- vs_currency (string, obrigatório) → Moeda de referência para preços (ex.: "usd", "eur", "brl").
- days (string ou int, obrigatório) → Período de dados a retornar. Pode ser um número de dias (ex.: 1, 30, 90, max) ou "max" para todo o histórico disponível.
- interval (string, opcional) → Intervalo de amostragem dos dados. Exemplos: "hourly", "daily". Se omitido, a API escolhe automaticamente conforme o período solicitado.

In [ ]:
cg.get_coin_market_chart_by_id(
    id="bitcoin", 
    vs_currency="usd", 
    days=7, 
    interval="daily")

####
```python
cg.get_coin_market_chart_range_by_id(id, vs_currency, from_timestamp, to_timestamp)
```

- id (string, obrigatório) → Identificador da moeda (ex.: "bitcoin", "ethereum").
- vs_currency (string, obrigatório) → Moeda de referência para preços (ex.: "usd", "eur", "brl").
- from_timestamp (int, obrigatório) → Timestamp UNIX inicial (em segundos) que define o começo do intervalo. Obs: É o número de segundos que se passaram desde 1º de janeiro de 1970, 00:00:00 UTC (o chamado Unix Epoch).
- to_timestamp (int, obrigatório) → Timestamp UNIX final (em segundos) que define o fim do intervalo.

In [ ]:
import time
# Setting UNIX timestamps (in seconds)
# January 1, 2025
from_timestamp = int(time.mktime(time.strptime('2025-01-01', '%Y-%m-%d')))
# January 10, 2025
to_timestamp = int(time.mktime(time.strptime('2025-01-10', '%Y-%m-%d')))

# Querying Bitcoin data within this range
btc_range = cg.get_coin_market_chart_range_by_id(
    id='bitcoin',
    vs_currency='usd',
    from_timestamp=from_timestamp,
    to_timestamp=to_timestamp
)

btc_range

####
```python
cg.get_coin_ohlc_by_id(id, vs_currency="usd", days="7")
```

- id (string, obrigatório) → Identificador da moeda (ex.: "bitcoin", "ethereum").
- vs_currency (string, obrigatório) → Moeda de referência para preços (ex.: "usd", "eur", "brl").
- days (string ou int, obrigatório) → Período de dados a retornar. Pode ser um número de dias (1, 7, 30) ou "max" para todo o histórico disponível.


Esse método retorna dados de OHLC (Open, High, Low, Close), ou seja, preços de abertura, máxima, mínima e fechamento da moeda no período solicitado.

In [ ]:
# Querying Bitcoin OHLC data for the last 7 days

btc_ohlc = cg.get_coin_ohlc_by_id(
    id='bitcoin',
    vs_currency='usd',
    days=7
)

# Each item returned is a list in the format:
# [time stamp -> time of measurement, open, high, low, closed]

# Example: show the first 3 records
for ohlc in btc_ohlc[:3]:
    print(ohlc)

[1762891200000, 103443.0, 103644.0, 102966.0, 103296.0]
[1762905600000, 103132.0, 103156.0, 102461.0, 102961.0]
[1762920000000, 103112.0, 103342.0, 102847.0, 103342.0]
